In [ ]:

import os
import cv2
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from skimage.feature import hog, local_binary_pattern

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter

In [ ]:
def load_and_preprocess_image(img_path, img_size=(256, 256)):
    """
    Load an image, convert to grayscale, resize, apply Gaussian blur,
    and normalize pixel values to [0,1].
    """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, img_size)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = img / 255.0
    return img

def extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        print(f"Warning: Unable to load image {image_path}")
        return None  # Skip this image
    
    image = cv2.resize(image, (128, 128))  # Resize for consistency
    
    # HOG Feature Extraction
    hog_features, _ = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
    
    # LBP Feature Extraction
    lbp = local_binary_pattern(image, P=24, R=3, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(27), density=True)
    
    return np.hstack((hog_features, lbp_hist))


def load_dataset(real_folder, fake_folder):
    X, y = [], []
    
    for file in os.listdir(real_folder):
        if file.lower() == "thumbs.db":
            continue
        img_path = os.path.join(real_folder, file)
        features = extract_features(img_path)
        if features is not None:
            X.append(features)
            y.append(1)
    
    for file in os.listdir(fake_folder):
        if file.lower() == "thumbs.db":
            continue
        img_path = os.path.join(fake_folder, file)
        features = extract_features(img_path)
        if features is not None:
            X.append(features)
            y.append(0)
    
    return np.array(X), np.array(y)


In [ ]:
# Load dataset
dataset_dir = './signature_dataset'
real_path = os.path.join(dataset_dir, "full_org")
fake_path = os.path.join(dataset_dir, "full_forg")
X, y = load_dataset(real_path, fake_path)

# Split dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train an SVM classifier (using RBF kernel)
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train)

# Evaluate classifier
y_pred = clf.predict(X_test)
print("=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred))
print("=== CONFUSION MATRIX ===")
print(confusion_matrix(y_test, y_pred))

# Save the model for later use
joblib.dump(clf, "svm_signature_classifier.pkl")

In [ ]:
# Cell 4: Setup for Neural Style Transfer (NST) and TensorBoard logging


# Set device and image size for NST
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 256  # Desired image size for NST

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter("runs/nst_signature")

# Define transforms for loading/unloading images (convert to 3-channel RGB for VGG)
loader = transforms.Compose([
    transforms.Resize(imsize),
    transforms.ToTensor()
])
unloader = transforms.ToPILImage()

def image_loader(image_path):
    """Load and preprocess an image for NST."""
    image = Image.open(image_path).convert("RGB")
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

def imshow(tensor, title=None):
    """Display a tensor as an image."""
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.show()


In [ ]:
# Cell 5: Define NST loss classes (Content and Style Loss) and helper functions

class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        # Detach target to avoid tracking history
        self.target = target.detach()

    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)
        return input

def gram_matrix(input):
    batch_size, feature_maps, h, w = input.size()
    features = input.view(batch_size * feature_maps, h * w)
    G = torch.mm(features, features.t())
    return G.div(batch_size * feature_maps * h * w)

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = nn.functional.mse_loss(G, self.target)
        return input

# Load pre-trained VGG19 for feature extraction
cnn = models.vgg19(pretrained=True).features.to(device).eval()
# Normalization parameters for VGG
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    """Module to normalize input images for VGG."""
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().detach().view(-1, 1, 1)
        self.std = std.clone().detach().view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=['conv_4'],
                               style_layers=['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']):
    """
    Build the model and attach style and content loss layers.
    """
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(normalization_mean, normalization_std).to(device)
    content_losses = []
    style_losses = []

    model = nn.Sequential(normalization)

    i = 0  # Increment count for conv layers
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f'conv_{i}'
        elif isinstance(layer, nn.ReLU):
            name = f'relu_{i}'
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f'pool_{i}'
        elif isinstance(layer, nn.BatchNorm2d):
            name = f'bn_{i}'
        else:
            raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')
        
        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    # Trim the model after the last loss layer
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:(i + 1)]

    return model, style_losses, content_losses


In [ ]:

#UNFINISHED LOTS OF PROBLEMS STILL FIGURING IT OUT!!


def load_artbench_bin(bin_file):
    """Load ArtBench-10 binary file."""
    with open(bin_file, 'rb') as f:
        raw_data = f.read()
    
    # Each image in ArtBench-10 is 32x32x3 (3072 bytes per image) + 1 byte for label
    img_size = 32 * 32 * 3  # Assuming 32x32 version
    record_size = img_size + 1  # 1 byte for label
    
    num_images = len(raw_data) // record_size  # Compute number of images
    images = np.zeros((num_images, 3, 32, 32), dtype=np.uint8)
    labels = np.zeros(num_images, dtype=np.uint8)
    
    for i in range(num_images):
        # Extract label (first byte of each record)
        offset = i * record_size
        labels[i] = raw_data[offset]
        
        # Extract image data (remaining bytes)
        img_start = offset + 1  # Skip label byte
        img_data = np.frombuffer(raw_data[img_start:img_start + img_size], dtype=np.uint8)
        images[i] = img_data.reshape(3, 32, 32)  # Reshape into (3, 32, 32)
    
    return images, labels


def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1e6, content_weight=1):
    """
    Run NST optimization to generate a synthetic signature.
    """
    # Initialize TensorBoard writer
    writer = SummaryWriter('./runs/style_transfer')
    
    # Check that all inputs are on the same device
    device = content_img.device
    
    # Make sure model and normalization values are on the same device
    normalization_mean = normalization_mean.to(device)
    normalization_std = normalization_std.to(device)
    
    # Get the style model and losses
    model, style_losses, content_losses = get_style_model_and_losses(
        cnn, normalization_mean, normalization_std, style_img, content_img)
    
    # Optimize only the image
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    
    run = [0]
    while run[0] <= num_steps:
        def closure():
            # Correct the values of updated input image
            with torch.no_grad():
                input_img.clamp_(0, 1)
                
            optimizer.zero_grad()
            model(input_img)
            
            style_score = sum(sl.loss for sl in style_losses)
            content_score = sum(cl.loss for cl in content_losses)
            
            loss = style_weight * style_score + content_weight * content_score
            loss.backward()
            
            # Log metrics to TensorBoard
            writer.add_scalar('Loss/Style', style_score.item(), run[0])
            writer.add_scalar('Loss/Content', content_score.item(), run[0])
            writer.add_scalar('Loss/Total', loss.item(), run[0])
            
            run[0] += 1
            return loss
        
        optimizer.step(closure)
    
    # Final correction
    with torch.no_grad():
        input_img.clamp_(0, 1)
        
    # Close TensorBoard writer
    writer.close()
    
    return input_img
content_dir = './signature_dataset/full_org/'
style_dir = './artbench10/'
save_dir = './synthetic_forged/'

os.makedirs(save_dir, exist_ok=True)
style_bin_files = [os.path.join(style_dir, f) for f in os.listdir(style_dir) if f.startswith('data_batch_') and f.endswith('.bin')]

# Load all ArtBench images into memory
style_images = np.concatenate([load_artbench_bin(f) for f in style_bin_files], axis=0)

for content_file in os.listdir(content_dir):
    if not content_file.endswith(('jpg', 'png')):
        continue
    
    content_img_path = os.path.join(content_dir, content_file)
    style_img_data = random.choice(style_images)  # Randomly select a style image
    
    style_img = torch.tensor(style_img_data / 255.0).float()
    transform = transforms.ToPILImage()
    style_img = transform(style_img)
    
    content_img = image_loader(content_img_path)
    input_img = content_img.clone()
    
    output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                content_img, style_img, input_img, num_steps=300)
    
    output_image = output.cpu().clone().squeeze(0)
    output_image = unloader(output_image)
    output_image.save(os.path.join(save_dir, content_file))
    
    print(f"Saved: {content_file} -> {save_dir}")


In [ ]:
# Cell 7: Evaluate the SVM classifier on the generated synthetic signature

# Read the synthetic image (as grayscale) for feature extraction
synthetic_img = cv2.imread("synthetic_signature.png", cv2.IMREAD_GRAYSCALE)
synthetic_img = cv2.resize(synthetic_img, (256, 256))
synthetic_img = synthetic_img / 255.0

# Extract features and reshape for the classifier
synthetic_features = extract_features(synthetic_img).reshape(1, -1)

# Predict using the pre-trained SVM classifier
prediction = clf.predict(synthetic_features)
probabilities = clf.predict_proba(synthetic_features)

print("=== CLASSIFIER PREDICTION ON SYNTHETIC SIGNATURE ===")
print("Prediction (0: real, 1: forged):", prediction[0])
print("Probabilities:", probabilities[0])
